In [1]:
from ipynb.fs.full.sim import Simulator
from ipynb.fs.full.traffic import Frame
from ipynb.fs.full.logger import Logger
from ipynb.fs.full.topology import SinkSwitch, BasicSwitch, Link, TrafficSource, ImpairedBasicSwitch
from ipynb.fs.full.frer import FRERSwitch, FRERMerge, FRERSplit
from ipynb.fs.full.ats import ATSGroup, ATSSwitch
import pandas as pd

### Test Case 1 ###
Simpelst Test Case 

- Adversarial frames
- FRER for 2 different paths, no delays, no drops, single ats group
- L = 2.5s
- T < 3L = 6s

In [2]:
sim = Simulator()
logger = Logger()
sink_logger = Logger()
ats_logger = Logger()

# Links
link_src1_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)
link_src2_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)
link_src3_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)

link_sw0_l1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)
link_sw0_s1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=0.5)

link_l1_sw1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)
link_s1_sw1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=0.5)

link_sw1_sw2 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)

link_sw2_sink = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)


# Sink
sink = SinkSwitch("SINK", sim, sink_logger)
sink_in = sink.add_ingress_port("in")
link_sw2_sink.connect(sink, sink_in)


# ATS
ats_groups = {"G1": ATSGroup(cir_bps=800, cbs_bytes=80, max_residence_time=6)}
routing_table = {"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]}
stream_to_group = {"Red": "G1", "Orange": "G1", "Blue": "G1"}

sw2 = ATSSwitch("ATS", sim, ats_logger, routing_table, stream_to_group, ats_groups)
ats_in = sw2.add_ingress_port("in")
link_sw1_sw2.connect(sw2, ats_in)
sw2.add_egress_port("out1", link_sw2_sink)
sw2.add_egress_port("out2", link_sw2_sink)
sw2.add_egress_port("out3", link_sw2_sink)


# FRER
split = FRERSplit({"Blue": ["out1", "out4"], "Red": ["out2"], "Orange": ["out3"]})

sw0 = FRERSwitch("SW_SPLIT", sim, logger, routing_table={}, frer_split=split)
split_in = sw0.add_ingress_port("in")
link_src1_sw0.connect(sw0, split_in)
link_src2_sw0.connect(sw0, split_in)
link_src3_sw0.connect(sw0, split_in)
sw0.add_egress_port("out1", link_sw0_l1)
sw0.add_egress_port("out2", link_sw0_l1)
sw0.add_egress_port("out3", link_sw0_l1)
sw0.add_egress_port("out4", link_sw0_s1)

merge = FRERMerge(expire_time=5.0)

sw1 = FRERSwitch("SW_MERGE", sim, logger, routing_table={"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]}, frer_merge=merge)
merge_in = sw1.add_ingress_port("in")
link_l1_sw1.connect(sw1, merge_in)
link_s1_sw1.connect(sw1, merge_in)
sw1.add_egress_port("out1", link_sw1_sw2)
sw1.add_egress_port("out2", link_sw1_sw2)
sw1.add_egress_port("out3", link_sw1_sw2)


# Switches
l1 = BasicSwitch("L1", sim, logger, routing_table={"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]})
l1_in = l1.add_ingress_port("in")
link_sw0_l1.connect(l1, l1_in)
l1.add_egress_port("out1", link_l1_sw1)
l1.add_egress_port("out2", link_l1_sw1)
l1.add_egress_port("out3", link_l1_sw1)

s1 = ImpairedBasicSwitch("S1", sim, logger, routing_table={"Blue": ["out"]}, impair_every_n=2, impair_mode="drop",)
s1_in = s1.add_ingress_port("in")
link_sw0_s1.connect(s1, s1_in)
s1.add_egress_port("out", link_s1_sw1)


# Sources
src1 = TrafficSource("BLUE", sim, logger)
src2 = TrafficSource("RED", sim, logger)
src3 = TrafficSource("ORANGE", sim, logger)
src1.add_egress_port("out", link_src1_sw0)
src2.add_egress_port("out", link_src2_sw0)
src3.add_egress_port("out", link_src3_sw0)

src1.start_periodic_loop(stream_id="Blue", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=0.0)
src2.start_periodic_loop(stream_id="Red", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=2.0)
src3.start_periodic_loop(stream_id="Orange", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=3.5)


sim.run(until=100)
with pd.option_context(
    "display.max_columns", None,
    "display.max_rows", None,
    "display.max_colwidth", None,
    "display.width", None,
):
    display(sink_logger.to_dataframe())
    display(ats_logger.to_dataframe())

,time,node,event,frame_id,stream_id,port
0,2.3,SINK,RECEIVE,1,Blue,in
1,2.3,SINK,SINK,1,Blue,in
2,5.3,SINK,RECEIVE,1,Red,in
3,5.3,SINK,SINK,1,Red,in
4,5.8,SINK,RECEIVE,2,Blue,in
5,5.8,SINK,SINK,2,Blue,in
6,6.8,SINK,RECEIVE,1,Orange,in
7,6.8,SINK,SINK,1,Orange,in
8,7.8,SINK,RECEIVE,2,Red,in
9,7.8,SINK,SINK,2,Red,in


,time,node,event,frame_id,stream_id,port
0,2.3,ATS,RECEIVE,1,Blue,in
1,2.3,ATS,FORWARD,1,Blue,out1
2,5.3,ATS,RECEIVE,1,Red,in
3,5.3,ATS,FORWARD,1,Red,out2
4,5.8,ATS,RECEIVE,2,Blue,in
5,5.8,ATS,FORWARD,2,Blue,out1
6,6.8,ATS,RECEIVE,1,Orange,in
7,6.8,ATS,FORWARD,1,Orange,out3
8,7.8,ATS,RECEIVE,2,Red,in
9,7.8,ATS,FORWARD,2,Red,out2


### Test Case 2 ### 
Test Case with drops for simples case

- Adversarial Frames 
- FRER for 2 different paths, every 2-nd frame on short path is droped, single ats group
- L = 2.5s
- T < 3L = 6s


In [3]:
sim = Simulator()
logger = Logger()
sink_logger = Logger()
ats_logger = Logger()

# Links
link_src1_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)
link_src2_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)
link_src3_sw0 = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)

link_sw0_l1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)
link_sw0_s1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=0.5)

link_l1_sw1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)
link_s1_sw1 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=0.5)

link_sw1_sw2 = Link(sim, bitrate_bps=800, transmission_delay_enabled=True, propagation_delay=1.0)

link_sw2_sink = Link(sim, bitrate_bps=800, transmission_delay_enabled=False, propagation_delay=0.0)


# Sink
sink = SinkSwitch("SINK", sim, sink_logger)
sink_in = sink.add_ingress_port("in")
link_sw2_sink.connect(sink, sink_in)


# ATS
ats_groups = {"G1": ATSGroup(cir_bps=800, cbs_bytes=80, max_residence_time=6)}
routing_table = {"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]}
stream_to_group = {"Red": "G1", "Orange": "G1", "Blue": "G1"}

sw2 = ATSSwitch("ATS", sim, ats_logger, routing_table, stream_to_group, ats_groups)
ats_in = sw2.add_ingress_port("in")
link_sw1_sw2.connect(sw2, ats_in)
sw2.add_egress_port("out1", link_sw2_sink)
sw2.add_egress_port("out2", link_sw2_sink)
sw2.add_egress_port("out3", link_sw2_sink)


# FRER
split = FRERSplit({"Blue": ["out1", "out4"], "Red": ["out2"], "Orange": ["out3"]})

sw0 = FRERSwitch("SW_SPLIT", sim, logger, routing_table={}, frer_split=split)
split_in = sw0.add_ingress_port("in")
link_src1_sw0.connect(sw0, split_in)
link_src2_sw0.connect(sw0, split_in)
link_src3_sw0.connect(sw0, split_in)
sw0.add_egress_port("out1", link_sw0_l1)
sw0.add_egress_port("out2", link_sw0_l1)
sw0.add_egress_port("out3", link_sw0_l1)
sw0.add_egress_port("out4", link_sw0_s1)

merge = FRERMerge(expire_time=5.0)

sw1 = FRERSwitch("SW_MERGE", sim, logger, routing_table={"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]}, frer_merge=merge)
merge_in = sw1.add_ingress_port("in")
link_l1_sw1.connect(sw1, merge_in)
link_s1_sw1.connect(sw1, merge_in)
sw1.add_egress_port("out1", link_sw1_sw2)
sw1.add_egress_port("out2", link_sw1_sw2)
sw1.add_egress_port("out3", link_sw1_sw2)


# Switches
l1 = BasicSwitch("L1", sim, logger, routing_table={"Blue": ["out1"], "Red": ["out2"], "Orange": ["out3"]})
l1_in = l1.add_ingress_port("in")
link_sw0_l1.connect(l1, l1_in)
l1.add_egress_port("out1", link_l1_sw1)
l1.add_egress_port("out2", link_l1_sw1)
l1.add_egress_port("out3", link_l1_sw1)

s1 = BasicSwitch("S1", sim, logger, routing_table={"Blue": ["out"]})
s1_in = s1.add_ingress_port("in")
link_sw0_s1.connect(s1, s1_in)
s1.add_egress_port("out", link_s1_sw1)


# Sources
src1 = TrafficSource("BLUE", sim, logger)
src2 = TrafficSource("RED", sim, logger)
src3 = TrafficSource("ORANGE", sim, logger)
src1.add_egress_port("out", link_src1_sw0)
src2.add_egress_port("out", link_src2_sw0)
src3.add_egress_port("out", link_src3_sw0)

src1.start_periodic_loop(stream_id="Blue", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=0.0)
src2.start_periodic_loop(stream_id="Red", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=2.0)
src3.start_periodic_loop(stream_id="Orange", frames_per_period=2, period=6, interval=2.5, size=10, count=2, offset=3.5)


sim.run(until=100)
with pd.option_context(
    "display.max_columns", None,
    "display.max_rows", None,
    "display.max_colwidth", None,
    "display.width", None,
):
    display(sink_logger.to_dataframe())
    display(ats_logger.to_dataframe())

,time,node,event,frame_id,stream_id,port
0,2.3,SINK,RECEIVE,1,Blue,in
1,2.3,SINK,SINK,1,Blue,in
2,4.8,SINK,RECEIVE,2,Blue,in
3,4.8,SINK,SINK,2,Blue,in
4,5.3,SINK,RECEIVE,1,Red,in
5,5.3,SINK,SINK,1,Red,in
6,6.8,SINK,RECEIVE,1,Orange,in
7,6.8,SINK,SINK,1,Orange,in
8,7.8,SINK,RECEIVE,2,Red,in
9,7.8,SINK,SINK,2,Red,in


,time,node,event,frame_id,stream_id,port
0,2.3,ATS,RECEIVE,1,Blue,in
1,2.3,ATS,FORWARD,1,Blue,out1
2,4.8,ATS,RECEIVE,2,Blue,in
3,4.8,ATS,FORWARD,2,Blue,out1
4,5.3,ATS,RECEIVE,1,Red,in
5,5.3,ATS,FORWARD,1,Red,out2
6,6.8,ATS,RECEIVE,1,Orange,in
7,6.8,ATS,FORWARD,1,Orange,out3
8,7.8,ATS,RECEIVE,2,Red,in
9,7.8,ATS,FORWARD,2,Red,out2
